Input format
==========
To demonstrate the python data types required to represent the input, we use the utility function ``make_data`` which returns randomly generated data that we will use for this example. See the function's documentation for more information regarding its parameters.

In [ ]:
from occuspytial.utils import make_data

Q, W, X, y, true_alpha, true_beta, true_tau, true_z = make_data(random_state=0)

The Gibbs samplers expect (at the minimum) as input the design matrix for occupancy covariates, the design matrices for detection covariates, the detection/non-detection data and the spatial precision matrix of the ICAR spatial random effects.

``Q`` represents the spatial precision matrix and can either be scipy's sparse matrix object or a numpy array. If a numpy array is used, the sampler convert it to a sparse matrix. It is advised that ``Q`` be in sparse format since the format is more memory efficient.

In [ ]:
Q 

Detection/non-detection observed data (``y``) should be depresented using a dictionary whose key is the site number (only sites that were surveyed) and value is a numpy array whose elements are 1's (if the species is detected on that visit) and 0's. The length of the array should represent the number of visits for that site.

In [ ]:
y.keys()

In [ ]:
y[15]  # 4 visits in site 15 with no detection of the species on

Similarly, the detection covariates (``W``) are represented by a dictionary whose key is the site number (only sites that were surveyed) and value is a 2d numpy array representing the design matrix of the detection covariates of that site.

In [ ]:
W[15]

Occupancy covariates (``X``) are represented by a design matrix which is a 2d numpy array.

In [ ]:
X[:5]  # occupancy design matrix for the first 5 sites

The rest of the output from ``make_data`` are the simulated true values for spatial occupancy model parameters.

In [ ]:
print('alpha: ', true_alpha, '\n', 'beta: ', true_beta, '\n', 'tau: ', true_tau, '\n', 'z (occupancy state): ', true_z)

Sampling example
===============

This section contains basic examples of how to use the available samplers.

Gibbs sampling
----------------------
Here we show how to use the Gibbs sampler presented in [Clark & Altwegg (2019)](https://onlinelibrary.wiley.com/doi/full/10.1002/ece3.4850) using OccuSpytial's sampler API. The name of the class implementing this sampler is ``LogitRSRGibbs``

In [ ]:
import numpy as np

from occuspytial import LogitRSRGibbs
from occuspytial.utils import make_data

# generate fake data with 500 sites in total
Q, W, X, y, true_alpha, true_beta, true_tau, true_z = make_data(500, tau_range=(0.1, 0.5), random_state=0)

Lets print out the true values of the parameters of interest.

In [ ]:
print('alpha: ', true_alpha, '\n', 'beta: ', true_beta, '\n', 'tau: ', true_tau)

In [ ]:
rsr = LogitRSRGibbs(Q, W, X, y, random_state=10)
rsr_samples = rsr.sample(2000, burnin=1000, chains=2)
# The progressbar is on by default, but Jupyter notebook only displays it on the console
# so it is not visible in the output cell of this notebook unlike if we are working on the console.

The output of the sample method is an instance of the ``PosteriorParameter`` class and inference of the samples obtained are done via the instance stored in the variable ``rsr_samples``. We can display the summary table using the `summary` attribute.

In [ ]:
rsr_samples.summary

To access the individual parameters we can use the the parameter's name as the key.

In [ ]:
rsr_samples['alpha']

Since we generated 2 chains, ``alpha`` parameter array is three dimensional; where the first dimension is the chain index,
the second dimension is the length of the post-burnin samples, and the third dimension is the size of the parameter (in elements).

In [ ]:
rsr_samples['alpha'].shape

``LogitRSRGibbs`` expects the prior distributions for ``alpha`` and ``beta`` to be normal distributed, and the prior
for ``tau`` to be Gamma distributed. We can pass custom values for the hyperparameters of these priors through the `hparams` dictionary parameter during instantiation of the class instance. More details on legal keys and values can be found in the docstring of the class.

In [ ]:
a_size = true_alpha.shape[0]
b_size = true_beta.shape[0]
hypers = {
    'a_mu': np.ones(a_size),  # alpha mean is an array of 1's
    'a_prec': np.eye(a_size) / 1000,  # alpha precision matrix (inverse of covariance) is diagonal matrix with entries (1/1000)
    'b_mu': np.ones(b_size),
    'b_prec': np.eye(a_size) / 1000,
    'tau_rate': 2,  # tau's rate parameter for the prior gamma distribution
    'tau_shape': 2,  # tau's shape parameter
}
rsr_hp = LogitRSRGibbs(Q, W, X, y, hparams=hypers, random_state=10)
rsrhp_samples = rsr_hp.sample(2000, burnin=1000, chains=2)
rsrhp_samples.summary

As we can see the MCMC chain did not converge in just 2000 samples with the provided hyper-parameter values. We can either
improve the estimates by using a much longer chain or use better starting values in the ``sample`` method via the `start` parameter.

Visualizing posterior samples
========================

``rsr_samples`` instance allows for convenient visualization of posterior samples of the parameters of interest. This functionality uses
[arviz](https://arviz-devs.github.io/arviz/index.html) as a backend. We can plot the traces and densities as follows:

In [ ]:
rsr_samples.plot_trace()

Parameters can be passed into the ``plot_trace`` method to configure the output plot. See arviz's [API reference](https://arviz-devs.github.io/arviz/generated/arviz.plot_trace.html#arviz.plot_trace) for valid input. Similarly, the Effective Sample Size (ESS) can be visualized as follows:

In [ ]:
rsr_samples.plot_ess()

For more information about other available plots, see documentation of ``PosteriorParameter`` class.